In [1]:
import os
import sys

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch3d
from pytorch3d import _C
from pytorch3d.io import load_obj, save_obj, load_objs_as_meshes
from pytorch3d.structures import Meshes, Pointclouds
from pytorch3d.ops import sample_points_from_meshes, knn_points, estimate_pointcloud_normals, knn_gather, cubify
from pytorch3d.loss.point_mesh_distance import point_mesh_edge_distance, point_mesh_face_distance
import trimesh

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import pyvista as pv
pv.start_xvfb()
pv.set_jupyter_backend('html')


from torch.utils.data import DataLoader, Dataset, DistributedSampler  
from torch.nn.parallel import DistributedDataParallel as DDP  

# # from pytorch3d.structures import Meshes

# from .utils import one_hot_sparse
from ops.mesh_geometry import *
# import pymeshlab as ml

import mcubes

In [2]:

trimesh_tem = trimesh.load('./data_example/Cocotree.obj', force='mesh')
# trimesh_tem = trimesh.load('/home/yihao/Document/Dataset3D/Train/person_0004.obj', force='mesh')

# delete a few faces to make the mesh non-watertight
# trimesh_tem.faces[590:593,:] = trimesh_tem.faces[590:591,:]
print(trimesh_tem.is_watertight)

device = torch.device("cuda")

mesh_tem = Meshes(verts=[torch.from_numpy(trimesh_tem.vertices).float()], 
                  faces=[torch.from_numpy(trimesh_tem.faces).long()]).to(device)

mesh_tem = normalize_mesh(mesh_tem) # normalize the mesh to fit in the unit cube

print(mesh_tem.verts_packed().shape)


False
torch.Size([1065, 3])


In [3]:
pl = pv.Plotter(notebook=True)

trimesh_tem = trimesh.Trimesh(vertices=mesh_tem.verts_list()[0].cpu().numpy(), faces=mesh_tem.faces_list()[0].cpu().numpy())

pl.add_mesh(trimesh_tem, color='lightblue', show_edges=True, opacity=1)


# pl.add_points(coordinates_downsampled.cpu().numpy()[np.random.choice(n_points, 1000)], color='red', point_size=5)

# pl.camera.roll = 10
pl.camera.elevation = 140
pl.camera.azimuth = 60
pl.camera.zoom = 1.3

pl.show() #screenshot='out_exp/tem_mesh_2.png', window_size=[800,800])

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Fipped duplication

In [4]:
## non-closed mesh check

centroids = mesh_tem.verts_packed()[mesh_tem.faces_packed()].mean(-2)

with torch.no_grad():
    #sdf_result = signed_distance_field(mesh_tem, coordinates_downsampled, allow_grad=False)
    occp_result = occupancy(mesh_tem, centroids, allow_grad=False)
    occp_result = occp_result.view(-1)

non_closed_index = torch.where((occp_result- torch.round(occp_result)).abs() > 1e-5)[0]

## duplicate & flip the non-closed faces 

mesh_verts_new = mesh_tem.verts_padded().clone() - mesh_tem.verts_normals_padded()*5*1e-3

B = mesh_verts_new.shape[0]

# faces_new = mesh_tem.faces_packed().clone()[non_closed_index,:]
faces_new = mesh_tem.faces_padded().clone()
faces_new = faces_new[:,:,[0,2,1]]

print(faces_new.shape)

# new_mesh = Meshes(verts=[mesh_verts_new], faces=[face_new])

# merge the old and new mesh
new_mesh = Meshes(verts=[mesh_verts_new[i] for i in range(B)], faces=[faces_new[i] for i in range(B)])

torch.Size([1, 1328, 3])


In [5]:
# voxelizer = Differentiable_Voxelizer(bbox_density=128)


sample_size = 512

meshbbox = mesh_tem.get_bounding_boxes()[0]

coordinates_downsampled = torch.stack(torch.meshgrid(torch.linspace(meshbbox[0,0], meshbbox[0,1], sample_size),
                                                        torch.linspace(meshbbox[1,0], meshbbox[1,1], sample_size),
                                                        torch.linspace(meshbbox[2,0], meshbbox[2,1], sample_size)), dim=-1)

coordinates_downsampled = coordinates_downsampled.view(-1, 3).to(device)

n_points = coordinates_downsampled.shape[0]


/home/yihao/anaconda3/envs/medicalimage/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [6]:
with torch.no_grad():
    #sdf_result = signed_distance_field(new_mesh, coordinates_downsampled, allow_grad=False)
    occp_result_ori = occupancy(mesh_tem, coordinates_downsampled, allow_grad=False, max_query_point_batch_size=2000)
    occp_result_inverse = occupancy(new_mesh, coordinates_downsampled, allow_grad=False, max_query_point_batch_size=2000)



In [7]:
occp_result = torch.where((occp_result_ori - occp_result_ori.round()).abs() > 1e-5, occp_result_inverse+occp_result_ori, occp_result_ori)
occpfield = occp_result.view(1, sample_size, sample_size, sample_size)
occpfield = occpfield.permute(0, 3, 2, 1)

Re-cubify

In [8]:
class GaussianBlur3D(nn.Module):
    def __init__(self):
        super(GaussianBlur3D, self).__init__()

    def forward(self, x, kernel_size=5, sigma=1.0):
        if kernel_size % 2 == 0:
            raise ValueError("Kernel size should be odd.")

        # Create Gaussian kernel
        kernel = self._create_gaussian_kernel(kernel_size, sigma)
        kernel = kernel.unsqueeze(0).unsqueeze(0)
        kernel = kernel.to(x.device)

        # Apply the convolution
        padding = kernel_size // 2
        return F.conv3d(x, kernel, padding=padding)

    def _create_gaussian_kernel(self, kernel_size, sigma):
        coords = torch.arange(kernel_size).float() - kernel_size // 2
        coords = coords.expand(kernel_size, kernel_size, kernel_size)
        g = torch.exp(-(coords**2 + coords.transpose(0,1)**2 + coords.transpose(0,2)**2) / (2*sigma**2))
        return g / g.sum()

# Create an instance of the 3D Gaussian Blur module
gaussian_blur_3d = GaussianBlur3D()

# Apply the 3D Gaussian filter with changeable kernel size
kernel_size = 3  # You can change this value
sigma = 2.0      # You can change this value
with torch.no_grad():
    occpfield_blurred = gaussian_blur_3d(occpfield, kernel_size=kernel_size, sigma=sigma)

In [9]:

# ## pymcubes marching cubes
# vertices, triangles = mcubes.marching_cubes((occpfield_blurred.permute(0, 3, 2, 1))[0].cpu().numpy(), 0.5)


# ## rescale the vertices to the original mesh
# vertices = meshbbox.mean(dim=-1).cpu().numpy() + (vertices*2/(sample_size-1)-1)*((meshbbox[:,1]-meshbbox[:,0]).view(1, 3).cpu().numpy())/2

# # create a mesh object
# trimesh_cubified = trimesh.Trimesh(vertices=vertices, faces=triangles)
# Pytorch3d cubify
cubified = cubify(occpfield_blurred, 0.2) # cubify the voxel grid, which is the inverse operation of voxelization



cubified = cubified.update_padded(meshbbox.mean(dim=-1) + cubified.verts_padded()*((meshbbox[:,1]-meshbbox[:,0]).view(1,1, 3).to(device))/2)

trimesh_cubified = trimesh.Trimesh(cubified.verts_packed().detach().cpu().numpy(), cubified.faces_packed().detach().cpu().numpy())


## smooth the mesh
# trimesh_cubified = trimesh.smoothing.filter_laplacian(trimesh_cubified, iterations=3)
pl = pv.Plotter(notebook=True)

pl.add_mesh(trimesh_tem, color='lightblue', show_edges=True, opacity=0.2)

pl.add_mesh(trimesh_cubified, color='lightgreen', opacity=1, show_edges=False)

pl.camera.elevation = 140
pl.camera.azimuth = 60
pl.camera.zoom = 1.3

pl.show() 


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…